In [1]:
# Class ETL Project for Stock Prices
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup as bs
import requests
import datetime as dt

In [2]:
# Read in the big historical price data file
stock_file = "Resources/historical_stock_prices.csv"
stock_data_df = pd.read_csv(stock_file)
stock_data_df.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [3]:
# Read in the stock industry sector data
sector_file = "Resources/historical_stocks.csv"
sector_data_df = pd.read_csv(sector_file)
sector_data_df.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [4]:
# Rename the columns as transform part one
sector_data_df = sector_data_df.rename(columns={"name": "firm_name"})
sector_data_df.head()

,ticker,exchange,firm_name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [5]:
# sort data on ticker to put in alpabetic order for sector data
sector_data_sorted_df = sector_data_df.sort_values('ticker')


In [6]:
sector_data_sorted_df.head()

,ticker,exchange,firm_name,sector,industry
3372,A,NYSE,"AGILENT TECHNOLOGIES, INC.",CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS
3385,AA,NYSE,ALCOA CORPORATION,BASIC INDUSTRIES,ALUMINUM
130,AABA,NASDAQ,ALTABA INC.,TECHNOLOGY,EDP SERVICES
3321,AAC,NYSE,"AAC HOLDINGS, INC.",HEALTH CARE,MEDICAL SPECIALITIES
150,AAL,NASDAQ,"AMERICAN AIRLINES GROUP, INC.",TRANSPORTATION,AIR FREIGHT/DELIVERY SERVICES


In [7]:
# sort data on ticker to put in alpabetic order for historical price data
stock_data_sorted_df = stock_data_df.sort_values(['ticker', 'date'])
stock_data_sorted_df.head()

,ticker,open,close,adj_close,low,high,volume,date
13766141,A,32.546494,31.473534,27.494957,28.612303,35.765381,62546300,1999-11-18
13766149,A,30.713520,28.880543,25.229753,28.478184,30.758226,15234100,1999-11-19
13766150,A,29.551144,31.473534,27.494957,28.657009,31.473534,6577800,1999-11-22
13766151,A,30.400572,28.612303,24.995413,28.612303,31.205294,5975600,1999-11-23
13766152,A,28.701717,29.372318,25.659359,28.612303,29.998211,4843200,1999-11-24


In [8]:
# Only use ticker, close and date columns from historical_stock_prices
stock_data_tickclosedate_df = stock_data_sorted_df[['ticker', 'close', 'date']].copy()
stock_data_tickclosedate_df.head()

,ticker,close,date
13766141,A,31.473534,1999-11-18
13766149,A,28.880543,1999-11-19
13766150,A,31.473534,1999-11-22
13766151,A,28.612303,1999-11-23
13766152,A,29.372318,1999-11-24


In [9]:
# obtain the year data of interest
stock_data_tickclosedate_df ['year'] = stock_data_tickclosedate_df['date'].str[:4]


In [10]:
stock_data_tickclosedate_df.head()

,ticker,close,date,year
13766141,A,31.473534,1999-11-18,1999
13766149,A,28.880543,1999-11-19,1999
13766150,A,31.473534,1999-11-22,1999
13766151,A,28.612303,1999-11-23,1999
13766152,A,29.372318,1999-11-24,1999


In [11]:
#stock_data_tickclosedate_year_df = stock_data_tickclosedate_df.filter(stock_data_tickclosedate_df['Year'] == '2018')
#stock_data_tickclosedate_df.head()
stock_data_tickclosedate_2018_df = stock_data_tickclosedate_df[stock_data_tickclosedate_df['year'] == '2018']
stock_data_tickclosedate_2018_df.head()


,ticker,close,date,year
13810939,A,67.599998,2018-01-02,2018
13810953,A,69.320000,2018-01-03,2018
13810959,A,68.800003,2018-01-04,2018
13810960,A,69.900002,2018-01-05,2018
13810961,A,70.050003,2018-01-08,2018


In [12]:
# First ticker to use is GM, PPG, AAPL, GENE, UPS, AB, YELP, ZN, LULU, ABB, EBAY, FE
sector_ticker_list = ['GM', 'PPG', 'AAPL', 'GENE', 'UPS', 'AB', 'YELP', 'ZN', 'LULU', 'ABB', 'EBAY', 'FE']

In [13]:
# execute call to web site within for loop, parse data needed and store in ticker_current_data
# first setup list to hold the data elements, then get current time, reformat to yyyy-mm-dd
ticker_current_data = []
date = dt.datetime.now()
date = date.strftime("%Y-%m-%d")

# loop over sector_ticker_list and obtain current price, store in dictionary with append method
for ticker in sector_ticker_list:
    # create URL for Motley fool quote 
    url_ticker = f"https://www.fool.com/quote/{ticker}"
    # obtain data from web site and store results in response
    response = requests.get(url_ticker)
    # parse response using standard html parser from Beautiful Soup
    soup = bs(response.text, 'html.parser')
    # find the data element current price using tag class and text current-price
    current_price = soup.find('span', class_='current-price').text.strip()
    # append to ticker_current_data, data elements, ticker, current_price, date to match table names
    ticker_current_data.append([ticker,current_price, date])

In [14]:
# print out results of web search and scrape
print(ticker_current_data)

[['GM', '$23.93', '2020-05-10'], ['PPG', '$91.98', '2020-05-10'], ['AAPL', '$310.13', '2020-05-10'], ['GENE', '$2.11', '2020-05-10'], ['UPS', '$94.83', '2020-05-10'], ['AB', '$22.80', '2020-05-10'], ['YELP', '$23.45', '2020-05-10'], ['ZN', '$0.30', '2020-05-10'], ['LULU', '$237.99', '2020-05-10'], ['ABB', '$18.33', '2020-05-10'], ['EBAY', '$42.15', '2020-05-10'], ['FE', '$40.60', '2020-05-10']]


In [15]:
# create a data frame to hold the scraped data, then rename the columns to match table names in database
# use datetime library
stock_scrape_data_df = pd.DataFrame(ticker_current_data)
stock_scrape_data_df = stock_scrape_data_df.rename(columns={0: "ticker", 1: "current_price", 2: "date"})


In [16]:
# check that the rename worked
stock_scrape_data_df

,ticker,current_price,date
0,GM,$23.93,2020-05-10
1,PPG,$91.98,2020-05-10
2,AAPL,$310.13,2020-05-10
3,GENE,$2.11,2020-05-10
4,UPS,$94.83,2020-05-10
5,AB,$22.80,2020-05-10
6,YELP,$23.45,2020-05-10
7,ZN,$0.30,2020-05-10
8,LULU,$237.99,2020-05-10
9,ABB,$18.33,2020-05-10


In [17]:
# save data to database tables - first establish the connection to stockprice database in postgres
rds_connection_string = "postgres:postgres@localhost:5432/stockprice"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
# check for the table names
engine.table_names()

['stock_scrape_data', 'hist_stock_prices', 'stock_indust_sector']

In [20]:
# Load the sector data into the database
sector_data_sorted_df.to_sql(name='stock_indust_sector', con=engine, if_exists='append', index=False)

In [23]:
# Load the scraped data into the data base
stock_scrape_data_df.to_sql(name='stock_scrape_data', con=engine, if_exists='append', index=False)

In [24]:
# save the converted stock data frame to an ascii file
# Export file as a CSV, without the Pandas index, but with the header
stock_data_tickclosedate_2018_df.to_csv("Resources/StockPriceSubset.csv", index=False, header=True)

In [25]:
# load the basic ascii data into database for historical purposes
stock_data_df.to_sql(name='hist_stock_prices', con=engine, if_exists='append', index=False)

In [ ]:
# write some select statements using the ticker to join between the 3 tables